In [1]:
import pandas as pd

In [2]:
!pip install arxiv

In [5]:
# creat a wrapper top of wekipdia
from langchain_community.tools import WikipediaQueryRun

# wrapper class
from langchain_community.utilities import WikipediaAPIWrapper

In [15]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper= api_wrapper)

In [16]:
wiki.name
# tool name is wikipedia

'wikipedia'

### Web based loader

In [10]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
document = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)
vectordb = FAISS.from_documents(document, OpenAIEmbeddings())

# its an intereface which retrive the result from the vector database
retriever = vectordb.as_retriever() 
retriever


VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DF96EB9EA0>)

### Create retriever tool

In [11]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, 'langsmith_search', 
                                       "Search for infromation about Langsmith. For any question about Langsmith, you must used this tools")

In [12]:
retriever_tool.name

'langsmith_search'

#### Creating Research paper wrapper 

In [14]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max = 200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name


'arxiv'

In [17]:
tools = [wiki, arxiv, retriever_tool]

In [18]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Generative_AI_Projects\\All_Genai_Project\\venv_lang_all_in_1\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for infromation about Langsmith. For any question about Langsmith, you must used this tools', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001DF94B657E0>, retriever=VectorStoreRetriever(tags=['FAI

## cretae llm

In [28]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-3.5-turbo-0125", temperature = 0)

### Taking generic propmts from lagnchain hub

In [29]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages 

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

## Create agents

In [30]:
from langchain.agents import  create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

## Agent Executor

In [33]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent, tools = tools , verbose = True)
agent_executor



AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [34]:
agent_executor.invoke({"input": "Tell me about Langsmith"})



> Entering new AgentExecutor chain...



Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace​We provide multiple ways to log traces to LangS

{'input': 'Tell me about Langsmith',
 'output': 'LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. LangSmith works independently and does not require the use of LangChain. Here are the steps to get started with LangSmith:\n\n1. Install LangSmith:\n   - Python: `pip install -U langsmith`\n   - TypeScript: `yarn add langchain langsmith`\n\n2. Create an API key:\n   - Head to the Settings page and click on Create API Key.\n\n3. Set up your environment:\n   - Set environment variables:\n     ```\n     export LANGCHAIN_TRACING_V2=true\n     export LANGCHAIN_API_KEY=<your-api-key>\n     export OPENAI_API_KEY=<your-openai-api-key>\n     ```\n\n4. Log your first trace:\n   - LangSmith provides multiple ways to log traces. You can use the provided code snippets to log your first trace.\n\nFor more detailed information and tutorials, you c

In [35]:
agent_executor.invoke({"input": "tell me about salman khan"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Salman Khan`


Page: Salman Khan
Summary: Abdul Rashid Salim Salman Khan (pronounced [səlˈmɑːn xɑːn]; born 27 December 1965) is an Indian actor, film producer, and television personality who works predominantly in HSalman Khan, whose full name is Abdul Rashid Salim Salman Khan, is an Indian actor, film producer, and television personality. He was born on December 27, 1965. Salman Khan is well-known for his work in the Indian film industry.

> Finished chain.


{'input': 'tell me about salman khan',
 'output': 'Salman Khan, whose full name is Abdul Rashid Salim Salman Khan, is an Indian actor, film producer, and television personality. He was born on December 27, 1965. Salman Khan is well-known for his work in the Indian film industry.'}

In [36]:
agent_executor.invoke({'input':'tell me about this research paper 1605.08386'})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThe research paper with the identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" authored by Caprice Stanley and Tobias Windisch. The paper, published on May 26, 2016, discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.

> Finished chain.


{'input': 'tell me about this research paper 1605.08386',
 'output': 'The research paper with the identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" authored by Caprice Stanley and Tobias Windisch. The paper, published on May 26, 2016, discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.'}